In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"

In [37]:
# get all models 
models = RootModel.get_submodels()
models.append(RootModel)

# Print Model Inheritance Tree

In [38]:
def print_inheritance_tree(root=None, tabs=0):
    
    if not root:
        root = RootModel
        
    print('\t'*tabs, root.__name__)
    submodels = root.__subclasses__()
    for model in submodels: 
        subsub_models = model.__subclasses__()
        if subsub_models: 
            print_inheritance_tree(model, tabs=tabs+1)
        else:
            print('\t'*(tabs+1), model.__name__)
        
   
    

In [39]:
print_inheritance_tree()

 RootModel
	 RootEntity
		 Person
		 MixinPerson
		 TempWork
	 TempMixin
		 MixinPerson
		 TempWork
	 StatementTemporalisationMixin
		 MixinPerson


# Print Model Subclasses

In [45]:
for m in models:
    submodels =[sc.__name__ for sc in m.__subclasses__()]
    print(m.__name__, submodels)


StatementTemporalisationMixin ['MixinPerson']
TempMixin ['MixinPerson', 'TempWork']
RootEntity ['Person', 'MixinPerson', 'TempWork']
Person []
MixinPerson []
TempWork []
RootModel ['RootEntity', 'TempMixin', 'StatementTemporalisationMixin']


# Show Leaf Models of Root Model

- these should be all models without submodels, i.e. all models above with an empty list

In [46]:
RootModel.get_leaf_models()

{example_one.models.MixinPerson,
 example_one.models.Person,
 example_one.models.TempWork}

# Models defined in example and their field inheritance

Inherited fields are accessable in child models and act like all other fields, but in the DB layer are stored and managed in the model's table that defines them (i.e. the models table that the child model inherited the field from)

In [49]:

for m in models:
    print(m.__name__)
    for field in m._meta.fields:
        if field.model == m:
            tag = "not inherited"
        else:
            tag = "inherited from: "
        print(f"\t {field.name} --> {tag} {field.model.__name__ if tag =='inherited from: ' else ''}")

StatementTemporalisationMixin
	 id --> inherited from:  RootModel
	 statement_start --> not inherited 
	 statement_end --> not inherited 
	 statement_temproalistaion_mixin_rootmodel_ptr --> not inherited 
TempMixin
	 id --> inherited from:  RootModel
	 start_date_written --> not inherited 
	 end_date_written --> not inherited 
	 tempmixin_rootmodel_ptr --> not inherited 
RootEntity
	 id --> inherited from:  RootModel
	 name --> not inherited 
	 rootentity_rootmodel_ptr --> not inherited 
Person
	 id --> inherited from:  RootModel
	 name --> inherited from:  RootEntity
	 rootentity_rootmodel_ptr --> inherited from:  RootEntity
	 first_name --> not inherited 
	 person_rootentity_ptr --> not inherited 
MixinPerson
	 start_date_written --> inherited from:  TempMixin
	 end_date_written --> inherited from:  TempMixin
	 tempmixin_rootmodel_ptr --> inherited from:  TempMixin
	 statement_start --> inherited from:  StatementTemporalisationMixin
	 statement_end --> inherited from:  StatementTempo

# Show only fields defined in given model

list of all fields defined per model, excluding inheritance one-to-one fields.

In [54]:
for m in models:
    fields = [f.name for f in m.get_fields() if f.model ==m]
    print(m.__name__)
    for f in fields:
        print("\t", f)

StatementTemporalisationMixin
	 statement_start
	 statement_end
TempMixin
	 start_date_written
	 end_date_written
RootEntity
	 name
Person
	 first_name
MixinPerson
	 first_name
	 new_field
TempWork
	 title
	 author
RootModel
	 id


# Show only fields that should be exposed to Users
- i.e. exclude the fields that where created to deal with inheritance
- shows all fields accessible in each model, regardless if they are inherited or not

In [58]:
for m in models:
    fields = m.get_fields()
    print(m.__name__)
    for f in fields:
        print("\t", f.name)

StatementTemporalisationMixin
	 id
	 statement_start
	 statement_end
TempMixin
	 id
	 start_date_written
	 end_date_written
RootEntity
	 id
	 name
Person
	 id
	 name
	 first_name
MixinPerson
	 start_date_written
	 end_date_written
	 statement_start
	 statement_end
	 id
	 name
	 first_name
	 new_field
TempWork
	 id
	 name
	 start_date_written
	 end_date_written
	 title
	 author
RootModel
	 id
